In [1]:
import pandas as pd
import numpy as np

from similarity import cosSimilarity,JaccardSimilarity,L2Similarity
from utils import read_data

arts,groupedArts,groups,label = read_data()

NO_ARTS = 1000
NO_GROUPS = 20

In [6]:
def bfNN(vec,data,label,metric=cosSimilarity):
    ret = None
    sim =0
    for idx, x in enumerate(data):
        curSim = metric(vec,x)
        if sim <curSim:
            ret = label[idx]
            sim = curSim
    return ret

In [3]:

from LSH import LSH
from itertools import chain
def LSHNNClassifier(b,r,data,label):

    lsh = LSH(data.shape[1],b,r)
    lsh.putMat(data)

    def labelVec(vec,id):
        lshIds = lsh.get(vec)
        #print(lshIds)
        ids = np.unique(list(chain.from_iterable(lshIds)))
        other =  [e for e in ids if e != id]

        return bfNN(vec,data[other],label[other])
    
    return [labelVec(data[i],i) for i in range(data.shape[0])]



In [4]:
import time
def measureRunTime(fun):
    start = time.time()
    ans = fun()
    return ans,time.time() - start

In [10]:
def test_d(d,r):
    tag,elapsed = measureRunTime(lambda : LSHNNClassifier(d,r,arts,label))
    error = sum([tag[i] != label[i] for i in range(len(label))]) / len(label)
    print("d = {},error = {}, time = {}".format(d,error,elapsed))
    return error

In [13]:
%%time
test_d(d,64)